## Load data

In [83]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
random_state = 12041500

In [84]:
def load_data():
    df_train = pd.read_json("./data/trainset.json")
    df_test = pd.read_json("./data/testset.json")

    df_train.drop(columns=['fnlwgt'], inplace=True)
    df_test.drop(columns=['fnlwgt'], inplace=True)

    return df_train, df_test

df_train, df_test = load_data()

ratio_features = ["age", "capital-gain", "capital-loss", "hours-per-week"]
ordinal_features = ["education-num"]
nominal_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
target = 'income'

## Train baseline

In [85]:
from sklearn.linear_model import LogisticRegression
from utils import create_model, train_and_evaluate, describe_model

In [86]:
clf = LogisticRegression(max_iter=1000, random_state=random_state)

In [87]:
## Train baseline
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_test, target, drop_na=True)
metrics = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.8477056548552315
Precision      0.8009474218724006
Recall         0.7607582564719824
F1             0.777270198343837


## Fairness Evaluation

In [88]:
from utils import split_data
from utils_fairness import search_bias, calc_fairness_score, explain_detected_bias

In [89]:
X_train, y_train = split_data(df_train, target, drop_na=True)

In [90]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

In [91]:
privileged_subset, _ = search_bias(X_train, y_train, probs, 1, penalty=1)

In [92]:
print(privileged_subset)

({'capital-gain': [1409, 1424, 1455, 1506, 1797, 2105, 2174, 2202, 2228, 2290, 2329, 2346, 2354, 2407, 2414, 2463, 2580, 2597, 2635, 2653, 2829, 2885, 2907, 2936, 2961, 3137, 3273, 3325, 3411, 3432, 3456, 3464, 3471, 3674, 3781, 3818, 3908, 3942, 4064, 4101, 4416, 4508, 4650, 4865, 4931, 5013, 5455, 5721, 6360, 6497, 6723, 6767, 6849, 7443, 7978, 10566, 22040, 34095, 41310]}, 547.3998)


In [93]:
calc_fairness_score(df_train, privileged_subset[0].keys(), target, verbose=True)

Sensitive Attributes: ['capital-gain']

Empty DataFrame
Columns: [Group, Distance, Proportion, Counts, P-Value]
Index: []

Weighted Mean Statistical Distance: nan


In [94]:
explain_detected_bias(df_train, probs, target, privileged_subset[0])

Our detected privileged group has a size of 984, we observe 0.0 as the average probability of earning >50k, but our model predicts 0.2604


## Fairness Metrics

In [95]:
from utils_fairness import transform_to_bias_dataset, describe_fairness, scan_and_calculate_fairness, plot_fairness_metrics

In [96]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

764 Na rows removed!
202 Na rows removed!


In [117]:
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_train, target, drop_na=True)
metrics = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   -0.12179388555614121
average_odds_difference         0.1719641410226968
equal_opportunity_difference    0.6004080756013745
disparate_impact                0.6155134201070837
theil_index                     0.12170921644827831


In [118]:
df_fairness_metrics, priviliged_subsets = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
), {}

for i in [1e-17, 1e-10, 0.001, 0.01, 0.1, 0, 1, 5, 10, 25, 50, 100]:
    metrics, priv = scan_and_calculate_fairness(model, df_train, target, i)    
    df_fairness_metrics.loc[f"{i}"] = metrics.values()
    priviliged_subsets[f"{i}"] = priv

In [122]:
df_fairness_metrics

,statistical_parity_difference,average_abs_odds_difference,equal_opportunity_difference,disparate_impact,theil_index
1e-17,-0.066640,0.199234,0.600408,0.746289,0.121709
1e-10,-0.066640,0.199234,0.600408,0.746289,0.121709
0.001,-0.066640,0.199234,0.600408,0.746289,0.121709
0.01,-0.066640,0.199234,0.600408,0.746289,0.121709
0.1,-0.069909,0.197622,0.600408,0.737043,0.121709
0,-0.066640,0.199234,0.600408,0.746289,0.121709
1,-0.121794,0.171964,0.600408,0.615513,0.121709
5,-0.214702,0.126118,0.600408,0.475085,0.121709
10,-0.280192,0.093735,0.600408,0.409512,0.121709
25,-0.537335,-0.034031,0.600408,0.266332,0.121709


In [147]:
def calc_diff(data, expectations, target, subset):
    if len(subset) == 0:
        return {
            "count": 0,
            "expected_probability": np.NaN,
            "model_probability": np.NaN
        }

    _df = data.copy()
    _df["expectations"] = expectations.copy()

    _to_choose = _df[subset.keys()].isin(subset).all(axis=1)
    _to_choose = _df.loc[_to_choose]

    return {
        "count": len(_to_choose),
        "expected_probability": np.round(_to_choose[target].mean(), 4),
        "model_probability": np.round(_to_choose["expectations"].mean(), 4),
    }

In [125]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

df_probs_diff = pd.DataFrame(
    columns=[
        "count",
        "expected_probability",
        "model_probability",
    ]
)
for penalty, priv in priviliged_subsets.items():
    df_probs_diff.loc[penalty] = calc_diff(df_train, probs, target, priv[0])

In [126]:
df_probs_diff

,count,expected_probability,model_probability
1e-17,1165,0.0,0.2594
1e-10,1191,0.0,0.2590
0.001,1191,0.0,0.2590
0.01,1191,0.0,0.2590
0.1,1177,0.0,0.2582
0,1165,0.0,0.2594
1,984,0.0,0.2604
5,671,0.0,0.2698
10,513,0.0,0.2686
25,213,0.0,0.2538


In [18]:
df_fairness_metrics.to_json("./results/fairness_metrics_original.json")

## Binned Data

In [127]:
df_train, df_test = load_data()
nominal_features = nominal_features + ['age', 'hours-per-week', 'capital-gain', 'capital-loss']

In [128]:
import numpy as np
import pandas as pd

def clean_data(X: pd.DataFrame) -> pd.DataFrame:
    X = X.reset_index(drop=True)
    cols = list(X.columns)
    X[cols] = X[cols].replace([" ?"], np.nan)
    X = X.dropna()
    def strip_str(x):
        if isinstance(x, str):
            return x.strip()
        else:
            return x
    X = X.applymap(strip_str)
    # X["relationship"] = X["relationship"].replace(["Husband", "Wife"], "Married")
    X["hours-per-week"] = pd.cut(
        x=X["hours-per-week"],
        bins=[0.9, 25, 39, 40, 55, 100],
        labels=["PartTime", "MidTime", "FullTime", "OverTime", "BrainDrain"],
    )
    X.age = pd.qcut(X.age, q=5)
    X["capital-gain"] = pd.cut(
        x=X["capital-gain"],
        bins=[-1, 0, 5000, 10000, 50000, X["capital-gain"].max() + 1],
        labels=["NoGain", "LowGain", "MediumGain", "HighGain", "VeryHighGain"],
    )
    X["capital-loss"] = pd.cut(
        x=X["capital-loss"],
        bins=[-1, 0, 1000, 2000, 5000, 100000],
        labels=["NoLoss", "LowLoss", "MediumLoss", "HighLoss", "VeryHighLoss"],
    )

    return X

df_train = clean_data(df_train.dropna())
df_test = clean_data(df_test.dropna())

In [129]:
for col in df_train.columns:
    if df_train[col].dtype == "category":
        df_train[col] = df_train[col].astype("object")
        df_test[col] = df_test[col].astype("object")

### Train baseline

In [130]:
from sklearn.linear_model import LogisticRegression
from utils import create_model, train_and_evaluate, describe_model

In [131]:
clf = LogisticRegression(max_iter=1000, random_state=random_state)

In [132]:
## Train baseline
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_test, target, drop_na=True)
metrics = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.8526183756663531
Precision      0.8096755453959336
Recall         0.7656258130687443
F1             0.7835223861090438


### Fairness Evaluation

In [133]:
from utils import split_data
from utils_fairness import search_bias, calc_fairness_score, explain_detected_bias

In [134]:
X_train, y_train = split_data(df_train, target, drop_na=True)

In [135]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

In [136]:
privileged_subset, _ = search_bias(X_train, y_train, probs, 1, penalty=1)

In [137]:
print(privileged_subset)

({'hours-per-week': ['FullTime', 'MidTime', 'OverTime'], 'workclass': ['Federal-gov', 'Local-gov', 'Private'], 'relationship': ['Husband', 'Wife'], 'capital-loss': ['HighLoss']}, 19.2157)


In [138]:
calc_fairness_score(df_train, privileged_subset[0].keys(), target, verbose=True)

Sensitive Attributes: ['capital-loss', 'hours-per-week', 'relationship', 'workclass']

                     Group Distance  Proportion  Counts   P-Value
        Own-child, Private    0.230    0.117701    4509  0.00e+00
                 Own-child    0.229    0.150852    5779  0.00e+00
 NoLoss, OverTime, Husband   -0.305    0.109948    4212  0.00e+00
         OverTime, Husband   -0.324    0.119189    4566  0.00e+00
          Husband, Private   -0.195    0.267352   10242 4.94e-324
           NoLoss, Husband   -0.190    0.381895   14630 4.94e-324
NoLoss, Own-child, Private    0.230    0.114986    4405 4.94e-324
                   Husband   -0.208    0.407685   15618 4.94e-324
         NoLoss, Own-child    0.230    0.147224    5640 4.94e-324
  NoLoss, Husband, Private   -0.177    0.250568    9599 1.20e-315

Weighted Mean Statistical Distance: 0.14188452864128123


In [139]:
explain_detected_bias(df_train, probs, target, privileged_subset[0])

Our detected privileged group has a size of 90, we observe 0.2222 as the average probability of earning >50k, but our model predicts 0.5559


### Fairness Metrics

In [140]:
from utils_fairness import transform_to_bias_dataset, describe_fairness, scan_and_calculate_fairness, plot_fairness_metrics

In [141]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

In [142]:
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_train, target, drop_na=True)
metrics = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   -0.34679406112724614
average_odds_difference         -0.35125457169171503
equal_opportunity_difference    -0.33872148084373654
disparate_impact                0.3630313162968948
theil_index                     0.1177542234136661


In [143]:
df_fairness_metrics, priviliged_subsets = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
), {}

for i in [1e-17, 1e-10, 0.001, 0.01, 0.1, 0, 1, 5, 10, 25, 50, 100]:
    metrics, priv = scan_and_calculate_fairness(model, df_train, target, i)    
    df_fairness_metrics.loc[f"{i}"] = metrics.values()
    priviliged_subsets[f"{i}"] = priv

In [144]:
df_fairness_metrics

,statistical_parity_difference,average_abs_odds_difference,equal_opportunity_difference,disparate_impact,theil_index
1e-17,0.266412,0.358207,0.618180,655.640546,0.117754
1e-10,0.266384,0.358199,0.618180,655.371397,0.117754
0.001,0.266384,0.358199,0.618180,655.371397,0.117754
0.01,-0.285315,-0.384832,-0.388161,0.409693,0.117754
0.1,0.126427,0.175246,0.343735,2.724830,0.117754
0,0.266412,0.358207,0.618180,655.640546,0.117754
1,-0.346794,-0.351255,-0.338721,0.363031,0.117754
5,-0.424474,-0.311169,-0.279585,0.316125,0.117754
10,0.000000,0.000000,0.000000,1.000000,0.000000
25,0.000000,0.000000,0.000000,1.000000,0.000000


In [145]:
df_fairness_metrics.to_json("./results/fairness_metrics_original_cleaned.json")

In [148]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

df_probs_diff = pd.DataFrame(
    columns=[
        "count",
        "expected_probability",
        "model_probability",
    ]
)
for penalty, priv in priviliged_subsets.items():
    df_probs_diff.loc[penalty] = calc_diff(df_train, probs, target, priv[0])

In [149]:
df_probs_diff

,count,expected_probability,model_probability
1e-17,9829,0.0095,0.0210
1e-10,9826,0.0095,0.0210
0.001,9826,0.0095,0.0210
0.01,60,0.0667,0.4962
0.1,382,0.0681,0.2038
0,9829,0.0095,0.0210
1,90,0.2222,0.5559
5,203,0.4433,0.6139
10,0,NaN,NaN
25,0,NaN,NaN


## PDFed Data

In [151]:
df_train, df_test = load_data()
nominal_features = nominal_features + ['age', 'hours-per-week']

In [152]:
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde

def clean_data(X: pd.DataFrame) -> pd.DataFrame:
     # Function to transform numerical attributes into their PDF values
    def transform_to_pdf(data, numerical_columns, target):
        transformed_data = data.copy()
        for column in numerical_columns:
            if column not in ['sex', target]:
                kde = gaussian_kde(data[column].dropna())
                transformed_data[column] = kde(data[column])
        return transformed_data

    # Transforming the numerical attributes
    # X = X.drop(columns=["fnlwgt", "education"], errors="ignore")
    cols = list(X.columns)
    X[cols] = X[cols].replace([" ?"], np.nan)
    X = X.dropna()

    def strip_str(x):
        if isinstance(x, str):
            return x.strip()
        else:
            return x
    X = X.applymap(strip_str)

    # X["relationship"] = X["relationship"].replace(["Husband", "Wife"], "Married")
    X = transform_to_pdf(X,['age', 'hours-per-week', 'capital-gain', 'capital-loss'], target)

    return X

df_train = clean_data(df_train.dropna())
df_test = clean_data(df_test.dropna())

In [153]:
for col in df_train.columns:
    if df_train[col].dtype == "category":
        df_train[col] = df_train[col].astype("object")
        df_test[col] = df_test[col].astype("object")

### Train baseline

In [154]:
from sklearn.linear_model import LogisticRegression
from utils import create_model, train_and_evaluate, describe_model

In [155]:
clf = LogisticRegression(max_iter=1000, random_state=random_state)

In [156]:
## Train baseline
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_test, target, drop_na=True)
metrics = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.8140482910003136
Precision      0.7474219922851064
Recall         0.7703756597408014
F1             0.7571757743396905


### Fairness Evaluation

In [157]:
from utils import split_data
from utils_fairness import search_bias, calc_fairness_score, explain_detected_bias

In [158]:
X_train, y_train = split_data(df_train, target, drop_na=True)

In [159]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

In [160]:
privileged_subset, _ = search_bias(X_train, y_train, probs, 1, penalty=1)

In [161]:
print(privileged_subset)

({'capital-gain': [1.108469510907749e-06, 5.765883500686317e-06, 6.10361661605396e-06, 6.812113570428972e-06, 7.014401995094196e-06, 7.1738364190331214e-06, 7.194661828926502e-06, 7.660438928410234e-06, 7.947687124397542e-06, 8.667196401393174e-06, 8.825617371877361e-06, 8.867547263415679e-06, 8.974838428689815e-06, 9.017723059882917e-06, 9.383215882157025e-06, 9.392498484571787e-06, 9.466276197254861e-06, 9.603682302865236e-06, 1.0040343731611617e-05, 1.0775653765085864e-05, 1.0920757983701274e-05, 1.1258138748347011e-05, 1.170231032823149e-05, 1.3770732245764749e-05, 1.405480988072596e-05, 1.502609359416476e-05, 1.910843724102672e-05, 1.9325712488370405e-05, 2.1109643625328754e-05, 2.3619222072924518e-05, 2.6465815179851564e-05, 2.779276464904525e-05, 2.9315798274844017e-05, 3.351370696288232e-05, 0.00010547658630157628, 0.00012424286014113114]}, 34.1879)


In [162]:
calc_fairness_score(df_train, privileged_subset[0].keys(), target, verbose=True)

Sensitive Attributes: ['capital-gain']

Empty DataFrame
Columns: [Group, Distance, Proportion, Counts, P-Value]
Index: []

Weighted Mean Statistical Distance: nan


In [163]:
explain_detected_bias(df_train, probs, target, privileged_subset[0])

Our detected privileged group has a size of 785, we observe 0.0 as the average probability of earning >50k, but our model predicts 0.2671


### Fairness Metrics

In [164]:
from utils_fairness import transform_to_bias_dataset, describe_fairness, scan_and_calculate_fairness, plot_fairness_metrics

In [165]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

In [166]:
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_train, target, drop_na=True)
metrics = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.18843905929928836
average_odds_difference         0.3432843498456296
equal_opportunity_difference    0.6444372852233677
disparate_impact                15.792466154994136
theil_index                     0.10592275572368795


In [167]:
df_fairness_metrics, priviliged_subsets = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
), {}

for i in [1e-17, 1e-10, 0.001, 0.01, 0.1, 0, 1, 5, 10, 25, 50, 100]:
    metrics, priv = scan_and_calculate_fairness(model, df_train, target, i)    
    df_fairness_metrics.loc[f"{i}"] = metrics.values()
    priviliged_subsets[f"{i}"] = priv

In [168]:
df_fairness_metrics

,statistical_parity_difference,average_abs_odds_difference,equal_opportunity_difference,disparate_impact,theil_index
1e-17,0.194652,0.345737,0.644437,23.677000,0.105923
1e-10,0.197982,0.348011,0.644437,55.709036,0.105923
0.001,0.197982,0.348011,0.644437,55.709036,0.105923
0.01,0.194652,0.345737,0.644437,23.677000,0.105923
0.1,0.194497,0.345680,0.644437,23.425549,0.105923
0,0.194652,0.345737,0.644437,23.677000,0.105923
1,0.188439,0.343284,0.644437,15.792466,0.105923
5,-0.085205,-0.076636,-0.085341,0.695386,0.105923
10,0.000000,0.000000,0.000000,1.000000,0.000000
25,0.000000,0.000000,0.000000,1.000000,0.000000


In [169]:
df_fairness_metrics.to_json("./results/fairness_metrics_original_pdf.json")

In [ ]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

df_probs_diff = pd.DataFrame(
    columns=[
        "count",
        "expected_probability",
        "model_probability",
    ]
)
for penalty, priv in priviliged_subsets.items():
    df_probs_diff.loc[penalty] = calc_diff(df_train, probs, target, priv[0])

In [ ]:
df_probs_diff

,count,expected_probability,model_probability
1e-17,9829,0.0095,0.0210
1e-10,9826,0.0095,0.0210
0.001,9826,0.0095,0.0210
0.01,60,0.0667,0.4962
0.1,382,0.0681,0.2038
0,9829,0.0095,0.0210
1,90,0.2222,0.5559
5,203,0.4433,0.6139
10,0,NaN,NaN
25,0,NaN,NaN
